## Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision
import torch.autograd as autograd

import imp
import os
import sys
import math
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import utils.training as train_utils; imp.reload(train_utils)
import utils.plot as plot_utils; imp.reload(plot_utils)

import camvid_dataset as camvid
import joint_transforms

/home/bfortuner/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
DATA_PATH='data/'
RESULTS_PATH='results/'
WEIGHTS_PATH='models/'
CAMVID_PATH=DATA_PATH+'CamVid/'
PROJECT_NAME='tiramisu'

## Design

**FirstConvLayer**

* 3x3 Conv2D (pad=, stride=, in_chans=3, out_chans=48)

**DenseLayer**

* BatchNorm
* ReLU
* 3x3 Conv2d (pad=, stride=, in_chans=, out_chans=) - "no resolution loss" - padding included
* Dropout (.2)

**DenseBlock**

* Input = FirstConvLayer, TransitionDown, or TransitionUp
* Loop to create L DenseLayers (L=n_layers)
* On TransitionDown we Concat(Input, FinalDenseLayerActivation)
* On TransitionUp we do not Concat with input, instead pass FinalDenseLayerActivation to TransitionUp block

**TransitionDown**

* BatchNorm
* ReLU
* 1x1 Conv2D (pad=, stride=, in_chans=, out_chans=)
* Dropout (0.2)
* 2x2 MaxPooling

**Bottleneck**

* DenseBlock (15 layers)

**TransitionUp**

* 3x3 Transposed Convolution (pad=, stride=2, in_chans=, out_chans=)
* Concat(PreviousDenseBlock, SkipConnection) - from cooresponding DenseBlock on transition down

**FinalBlock**

* 1x1 Conv2d (pad=, stride=, in_chans=256, out_chans=n_classes)
* Softmax

**FCDenseNet103 Architecture**

* input (in_chans=3 for RGB)
* 3x3 ConvLayer (out_chans=48)
* DB (4 layers) + TD
* DB (5 layers) + TD
* DB (7 layers) + TD
* DB (10 layers) + TD
* DB (12 layers) + TD
* Bottleneck (15 layers)
* TU + DB (12 layers)
* TU + DB (10 layers)
* TU + DB (7 layers)
* TU + DB (5 layers)
* TU + DB (4 layers)
* 1x1 ConvLayer (out_chans=n_classes) n_classes=11 for CamVid
* Softmax

**FCDenseNet67**

* GrowthRate (k) = 16
* 5 layers per dense block
* 1 Conv Layer
* 5 DenseBlocks Downsample (25 layers)
* 5 TransitionDown
* 5 Bottleneck layers
* 5 Dense Blocks Upsample (25 layers)
* 5 TransitionUp
* 1 Conv Layer
* 1 Softmax layer (doesn't count)
67 Total layers

## Layers

In [3]:
class DenseLayer(nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        
        #author's impl - lasange 'same' pads with half 
        # filter size (rounded down) on "both" sides
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, 
                out_channels=growth_rate, kernel_size=3, stride=1, 
                  padding=1, bias=True))
        
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        return super(DenseLayer, self).forward(x)

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers, upsample=False):
        super(DenseBlock, self).__init__()
        self.upsample = upsample
        self.layers = nn.ModuleList([DenseLayer(
            in_channels + i*growth_rate, growth_rate)
            for i in range(n_layers)])
        
    def forward(self, x):
        if self.upsample:
            new_features = []
            #we pass all previous activations into each dense layer normally
            #But we only store each dense layer's output in the new_features array
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1)
                new_features.append(out)
            return torch.cat(new_features,1)
        else:
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1) # 1 = channel axis
            return x 
    
class TransitionDown(nn.Sequential):
    def __init__(self, in_channels):
        super(TransitionDown, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(in_channels=in_channels, 
              out_channels=in_channels, kernel_size=1, stride=1, 
                padding=0, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))
        self.add_module('maxpool', nn.MaxPool2d(2))
        
    def forward(self, x):
        return super(TransitionDown, self).forward(x)
    
class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionUp, self).__init__()
        self.convTrans = nn.ConvTranspose2d(in_channels=in_channels, 
               out_channels=out_channels, kernel_size=3, stride=2, 
                padding=0, bias=True) #crop = 'valid' means padding=0. Padding has reverse effect for transpose conv (reduces output size)
        #http://lasagne.readthedocs.io/en/latest/modules/layers/conv.html#lasagne.layers.TransposedConv2DLayer
    
    def forward(self, x):
        return self.convTrans(x)
    
class Bottleneck(nn.Sequential):
    def __init__(self, in_channels, growth_rate, n_layers):
        super(Bottleneck, self).__init__()
        self.add_module('bottleneck', DenseBlock(in_channels, growth_rate, n_layers, upsample=True))

    def forward(self, x):
        return super(Bottleneck, self).forward(x)

## Model

In [4]:
def center_crop(layer, target_width):
    #Assumes square
    batch_size, n_channels, layer_width, layer_height = layer.size()
    xy1 = (layer_width - target_width) // 2
    return layer[:, :, xy1:(xy1 + target_width), xy1:(xy1 + target_width)]

In [5]:
class FCDenseNet(nn.Module):
    def __init__(self, in_channels=3, n_blocks=5, layers_per_block=5, growth_rate=16, 
                 out_chans_first_conv=48, n_classes=12):
        super(FCDenseNet, self).__init__()
        self.n_blocks = n_blocks
        self.n_channels = 0
        self.skipNumChannels = []
        
        
        #####################
        # First Convolution #
        #####################

        self.add_module('firstconv', nn.Conv2d(in_channels=in_channels, 
                  out_channels=out_chans_first_conv, kernel_size=3, 
                  stride=1, padding=1, bias=True))
        self.n_channels = out_chans_first_conv
        
        
        
        #####################
        # Downsampling path #
        #####################
        
        self.denseBlocksDown = nn.ModuleList([])
        self.transDownBlocks = nn.ModuleList([])
        for i in range(n_blocks):
            self.denseBlocksDown.append(
                DenseBlock(self.n_channels, growth_rate, layers_per_block))
            self.n_channels += (growth_rate*layers_per_block)
            #print("nchannelsdensedown%d: %d" % (i, self.n_channels))
            self.skipNumChannels.insert(0,self.n_channels)
            self.transDownBlocks.append(TransitionDown(self.n_channels))
            
            
            
        #####################
        #     Bottleneck    #
        #####################
        
        self.add_module('bottleneck',Bottleneck(self.n_channels, 
                                     growth_rate, layers_per_block))
        prev_block_channels = growth_rate*layers_per_block
        self.n_channels += prev_block_channels 
        #print("bottleneck: %d" % (self.n_channels))
        
        
        
        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = nn.ModuleList([])
        self.denseBlocksUp = nn.ModuleList([])
        for i in range(n_blocks-1):
            self.transUpBlocks.append(TransitionUp(
                growth_rate*layers_per_block, growth_rate*layers_per_block))
            self.n_channels = prev_block_channels + self.skipNumChannels[i]
            #print("nchannelsTU%d: %d" % (i, self.n_channels))
            self.denseBlocksUp.append(DenseBlock(
                self.n_channels, growth_rate, layers_per_block, 
                    upsample=True))
            prev_block_channels = growth_rate*layers_per_block
            self.n_channels += prev_block_channels
            #print("nchannelsdensedown%d: %d" % (i, self.n_channels))
            
        #One final normal dense block
        self.transUpBlocks.append(TransitionUp(
            growth_rate*layers_per_block, growth_rate*layers_per_block))
        self.n_channels = prev_block_channels + self.skipNumChannels[-1]
        #print("nchannelsTU%d: %d" % (i, self.n_channels))
        self.denseBlocksUp.append(DenseBlock(
            self.n_channels, growth_rate, layers_per_block, 
                upsample=False))
        self.n_channels += growth_rate*layers_per_block

        
        
        #####################
        #      Softmax      #
        #####################

        self.finalConv = nn.Conv2d(in_channels=self.n_channels, 
               out_channels=n_classes, kernel_size=1, stride=1, 
                   padding=0, bias=True)
        self.softmax = nn.Softmax()
        
        
    def forward(self, x):
        print("INPUT SIZE",x.size())
        out = self.firstconv(x)
        
        skip_connections = []
        for i in range(self.n_blocks):
            out = self.denseBlocksDown[i](out)
            skip_connections.append(out)
            out = self.transDownBlocks[i](out)
            
        out = self.bottleneck(out)
        print("BOTTLESIZE", out.size())
        
        for i in range(self.n_blocks):
            out = self.transUpBlocks[i](out)
            skip = skip_connections.pop()
            out = center_crop(out, skip.size()[2])
            out = torch.cat([out, skip], 1)
            print(skip.size(), out.size())
            out = self.denseBlocksUp[i](out)
            print ("DENSEOUT", out.size())
            
        out = self.finalConv(out)
        print("LASTCONV", out.size())
        # Reshape
        bs, c, h, w = out.size()
        out = out.view(bs*h*w, c)
        
        out = self.softmax(out)
        return out

In [6]:
model = FCDenseNet()
print(model)

FCDenseNet (
  (firstconv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (denseBlocksDown): ModuleList (
    (0): DenseBlock (
      (layers): ModuleList (
        (0): DenseLayer (
          (norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (1): DenseLayer (
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (2): DenseLayer (
          (norm): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(80, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (3): DenseLayer (
          (no

## Config

* https://github.com/SimJeg/FC-DenseNet/blob/master/config/FC-DenseNet103.py

In [7]:
train_crop_size = (224, 224) # None for full size
n_classes = 12 #11 + background

# Training
seed = 0
learning_rate = .001
lr_sched_decay = 0.995 # Applied each epoch
weight_decay = 0.0001
num_epochs = 750
max_patience = 150
loss_function = 'crossentropy'
optimizer = 'rmsprop' # Consider adam for training on other dataset, or decrease epsilon to 1e-12
batch_size = 1

In [8]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        #kaiming is first name of author whose last name is 'He' lol
        init.kaiming_uniform(m.weight) 
        m.bias.data.zero_()

## Data

* https://github.com/SimJeg/FC-DenseNet/blob/master/data_loader.py
* https://github.com/pytorch/vision/pull/90
* https://github.com/SimJeg/FC-DenseNet/issues/10

In [9]:
torch.cuda.manual_seed(seed)

In [10]:
traindir = os.path.join(CAMVID_PATH, 'train')
valdir = os.path.join(CAMVID_PATH, 'val')
testdir = os.path.join(CAMVID_PATH, 'test')

normalize = transforms.Normalize(mean=camvid.mean, std=camvid.std)
train_joint_transformer = transforms.Compose([
    joint_transforms.JointRandomCrop(224),
    joint_transforms.JointRandomHorizontalFlip()
    ])
train_dset = camvid.CamVid(CAMVID_PATH, 'train',
      joint_transform=train_joint_transformer,
      transform=transforms.Compose([
          transforms.ToTensor(),
          normalize,
    ]))
train_loader = torch.utils.data.DataLoader(
    train_dset, batch_size=batch_size, shuffle=True)

val_dset = camvid.CamVid(
    CAMVID_PATH, 'val', joint_transform=None,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]))
val_loader = torch.utils.data.DataLoader(
    val_dset, batch_size=batch_size, shuffle=False)

test_dset = camvid.CamVid(
    CAMVID_PATH, 'test', joint_transform=None,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dset, batch_size=batch_size, shuffle=False)

In [11]:
# print(train_loader.dataset.classes)
# print(train_loader.dataset.class_weight)
# print(train_loader.dataset.imgs[:3])
# print(train_loader.dataset.mean)
# print(train_loader.dataset.std)
print("TrainImages: %d" %len(train_loader.dataset.imgs))
print("ValImages: %d" %len(val_loader.dataset.imgs))
print("TestImages: %d" %len(test_loader.dataset.imgs))
print("NumClasses: %d" % len(train_loader.dataset.classes))

TrainImages: 367
ValImages: 101
TestImages: 233
NumClasses: 12


In [12]:
inputs, targets = next(iter(train_loader))
t = targets[0]
i = inputs[0]

#Inputs are tensors of normalized pixel values
print (inputs[0].size())
print(i.max())
print(i.min())

#Targets are tensors of class labels from 0-11 (0 means background)
print(targets[0].size())
print(t.max())
print(t.min())

#Reshape Target To Match Output of Model
#360*480 = 172800
#targets[:, 0].size()
values, indices = t.cpu().max(0)

torch.Size([3, 224, 224])
2.145308017730713
-1.5297001600265503
torch.Size([224, 224])
11
0


In [13]:
#Test Model Output
model = FCDenseNet(in_channels=3, n_blocks=5, layers_per_block=5, growth_rate=16, 
                 out_chans_first_conv=48, n_classes=n_classes)

In [14]:
model = model.cuda()

In [15]:
model.apply(weights_init)

FCDenseNet (
  (firstconv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (denseBlocksDown): ModuleList (
    (0): DenseBlock (
      (layers): ModuleList (
        (0): DenseLayer (
          (norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (1): DenseLayer (
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (2): DenseLayer (
          (norm): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True)
          (relu): ReLU (inplace)
          (conv): Conv2d(80, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d (p=0.2)
        )
        (3): DenseLayer (
          (no

In [16]:
pred = model(Variable(inputs.cuda()))

INPUT SIZE torch.Size([1, 3, 224, 224])
BOTTLESIZE torch.Size([1, 80, 7, 7])
torch.Size([1, 448, 14, 14]) torch.Size([1, 528, 14, 14])
DENSEOUT torch.Size([1, 80, 14, 14])
torch.Size([1, 368, 28, 28]) torch.Size([1, 448, 28, 28])
DENSEOUT torch.Size([1, 80, 28, 28])
torch.Size([1, 288, 56, 56]) torch.Size([1, 368, 56, 56])
DENSEOUT torch.Size([1, 80, 56, 56])
torch.Size([1, 208, 112, 112]) torch.Size([1, 288, 112, 112])
DENSEOUT torch.Size([1, 80, 112, 112])
torch.Size([1, 128, 224, 224]) torch.Size([1, 208, 224, 224])
DENSEOUT torch.Size([1, 288, 224, 224])
LASTCONV torch.Size([1, 12, 224, 224])


In [19]:
tns = pred.data.cpu()

In [20]:
tns.size()

torch.Size([50176, 12])

## Visualize

* https://discuss.pytorch.org/t/convert-pixel-wise-class-tensor-to-image-segmentation/1268

In [18]:
Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
#Road_marking = [255,69,0] ???
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

label_colours = np.array([Sky, Building, Pole, Road, Pavement,
      Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

def view_annotated(tensor, plot=True):
    temp = tensor.numpy()
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,11):
        r[temp==l]=label_colours[l,0]
        g[temp==l]=label_colours[l,1]
        b[temp==l]=label_colours[l,2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:,:,0] = (r/255.0)#[:,:,0]
    rgb[:,:,1] = (g/255.0)#[:,:,1]
    rgb[:,:,2] = (b/255.0)#[:,:,2]
    if plot:
        plt.imshow(rgb)
    else:
        return rgb
    
def view_image(inp):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array(camvid.mean)
    std = np.array(camvid.std)
    inp = std * inp + mean
    plt.imshow(inp)
    plt.show()

In [62]:
# Get a batch of training data
inputs, targets = next(iter(train_loader))
#inputs, targets = next(iter(val_loader))
#inputs, targets = next(iter(test_loader))

# Plot Single Image
view_image(inputs[0])

In [63]:
# Plot Annotated Image
view_annotated(targets[0])

In [64]:
# Plot Grid of images
out = torchvision.utils.make_grid(inputs, nrow=3)
view_image(out)

## Train

* https://github.com/pytorch/examples/blob/master/imagenet/main.py

**Hyperparameters**

* WeightInitialization = HeUniform
* Optimizer = RMSProp
* LR = .001 with exponential decay of 0.995 after each epoch
* Data Augmentation = Random Crops, Vertical Flips
* ValidationSet with early stopping based on IoU or MeanAccuracy with patience of 100 (50 during finetuning)
* WeightDecay = .0001
* Finetune with full-size images, LR = .0001
* Dropout = 0.2
* BatchNorm "we use current batch stats at training, validation, and test time"

**CamVid**

* TrainingSet = 367 frames
* ValidationSet = 101 frames
* TestSet = 233 frames
* Images of resolution 360x480
* Images "Cropped" to 224x224 for training --- center crop?
* FullRes images used for finetuning
* NumberOfClasses = 11 (output)
* BatchSize = 3

**FCDenseNet103**

* GrowthRate = 16 (k, number of filters to each denselayer adds to the ever-growing concatenated output)
* No pretraining

In [67]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data.cuda()), Variable(target.cuda())
        optimizer.zero_grad()
        output = model(data)
        print (output.size(), target.size())
        loss = criterion(output, target) #.view(-1,n_classes))
        loss.backward()
        optimizer.step()

## Loss

* https://github.com/ycszen/pytorch-ss/blob/master/loss.py
* http://pytorch.org/docs/nn.html?highlight=logsoftmax#nllloss2d
* https://github.com/SimJeg/FC-DenseNet/blob/master/metrics.py

In [ ]:
import torch
import torch.nn.functional as F

def CrossEntropy2d(input, target, weight=None, size_average=False):
    # input:(n, c, h, w) target:(n, h, w)
    n, c, h, w = input.size()

    input = input.transpose(1, 2).transpose(2, 3).contiguous()
    input = input[target.view(n, h, w, 1).repeat(1, 1, 1, c) >= 0].view(-1, c)

    target_mask = target >= 0
    target = target[target_mask]
    #loss = F.nll_loss(F.log_softmax(input), target, weight=weight, size_average=False)
    loss = F.cross_entropy(input, target, weight=weight, size_average=False)
    if size_average:
        loss /= target_mask.sum().data[0]

    return loss

In [112]:
nn.NLLLoss2d??

In [111]:
class CrossEntropyLoss2d(nn.Module):
    def __init__(self, weight=None):
        super().__init__()

        self.loss = nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(F.log_softmax(outputs), targets)

In [142]:
n, c, h, w = inputs.size()
print(inputs.size())
rs = inputs.transpose(1, 2).transpose(2, 3).contiguous()
print(rs.size())

rs = rs[targets.view(n, h, w, 1).repeat(1, 1, 1, c) >= 0].view(-1, c)
print(rs.size())

torch.Size([3, 3, 224, 224])
torch.Size([3, 224, 224, 3])
torch.Size([150528, 3])


In [148]:
n, c, h, w = inputs.size()
out = inputs.view(n*h*w, c)
print(out.size())

torch.Size([150528, 3])


AttributeError: 'FloatTensor' object has no attribute 'equals'

In [ ]:
#2d version?
criterion = nn.CrossEntropyLoss(weight=camvid.class_weight.cuda()).cuda()

In [ ]:
#2d version?
criterion = CrossEntropyLoss2d(weight=camvid.class_weight.cuda()).cuda()

In [106]:
epochs=1
model = FCDenseNet(in_channels=3, n_blocks=5, layers_per_block=5, growth_rate=16, 
                 out_chans_first_conv=48, n_classes=n_classes)
model = model.cuda()
model.apply(weights_init)
#cudnn.benchmark = True ??

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


In [ ]:
train(model, train_loader, optimizer, criterion)

In [37]:
def adjust_learning_rate(optimizer, epoch, decay):
    """Decay LR by `decay` every 1 epochs"""
    lr = optimizer.param_groups[0]['lr']
    lr *= (decay ** (epoch // 1))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# Adjust patience
# https://github.com/SimJeg/FC-DenseNet/blob/master/train.py#L176

In [82]:
existing_weights_fpath=None
nEpochs=1

net = FCDenseNet(in_channels=3, n_blocks=5, layers_per_block=5, growth_rate=16, 
                 out_chans_first_conv=48, n_classes=11)
net = net.cuda()
cudnn.benchmark = True

optimizer = optim.RMSprop(net.parameters(), lr=.001, weight_decay=.0001)
criterion = nn.CrossEntropyLoss().cuda()

print('  + Number of params: {}'.format(
    sum([p.data.nelement() for p in net.parameters()])))

if existing_weights_fpath:
    startEpoch = train_utils.load_weights(net, existing_weights_fpath)
    endEpoch = startEpoch + nEpochs
    print ('Resume training at epoch: {}'.format(startEpoch))
    if os.path.exists(RESULTS_PATH+'train.csv'): #assume test.csv exists
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), append_write)
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), append_write)
else:
    print ("Training new model from scratch")
    startEpoch = 1
    endEpoch = nEpochs
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), 'w')
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), 'w')


for epoch in tqdm(range(startEpoch, endEpoch+1)):
    since = time.time()
    train_utils.adjust_opt("sgd", optimizer, epoch)
    train_utils.train(epoch, net, trainLoader, optimizer, trainF)
    train_utils.test(epoch, net, testLoader, optimizer, testF)
    time_elapsed = time.time() - since  
    print('Time {:.0f}m {:.0f}s\n'.format(
        time_elapsed // 60, time_elapsed % 60))
    if epoch != 1:
        os.system('./utils/plot.py {} &'.format(RESULTS_PATH))

trainF.close()
testF.close()


  0%|          | 0/1 [00:00<?, ?it/s]

  + Number of params: 1059298
Training new model from scratch


Epoch 1: Train - Loss: 1.370707	Error: 43.750000


100%|██████████| 1/1 [01:12<00:00, 72.19s/it]

Test - Loss: 1.3404, Error: 4590/10000 (46%)
Time 1m 12s



## References

* https://github.com/mattmacy/vnet.pytorch/blob/master/train.py
* 